In [ ]:
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
sns.set_style('whitegrid')
from sklearn.metrics import confusion_matrix , classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Flatten , Conv2D , MaxPooling2D , Dropout , Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam , Adamax
from tensorflow.keras import regularizers

#Warnings
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, ReLU, Add
from tensorflow.keras.models import Model
from tensorflow.keras.losses import KLDivergence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import DenseNet121, ResNet50V2

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, ReLU, Add
from tensorflow.keras.models import Model
from tensorflow.keras.losses import KLDivergence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import DenseNet169, MobileNetV2, ResNet50, EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D
import copy


from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import l2

from tensorflow.keras.constraints import MinMaxNorm
from sklearn.model_selection import train_test_split
from tensorflow.keras.initializers import HeNormal, RandomNormal


In [ ]:
pip install lifelines

In [ ]:
from lifelines.utils import concordance_index

### Modality 1

In [ ]:
X_train_h = np.load('ham10000/x_train.npy')
y_train_h = np.load('ham10000/y_train.npy')
X_test_h = np.load('ham10000/x_test.npy')
y_test_h = np.load('ham10000/y_test.npy')

X_val_h = np.load('ham10000/x_val.npy')
y_val_h = np.load('ham10000/y_val.npy')


X_train_h.shape, y_train_h.shape, X_test_h.shape, y_test_h.shape, X_val_h.shape, y_val_h.shape


In [ ]:
random_indices = np.random.choice(1001, 810, replace=False)

X_test_h1 = X_test_h[random_indices]
y_test_h1 = y_test_h[random_indices]

X_test_h1.shape, y_test_h1.shape, X_test_h.shape, y_test_h.shape

In [ ]:
random_indices = np.random.choice(1002, 648, replace=False)

X_val_h1 = X_val_h[random_indices]
y_val_h1 = y_val_h[random_indices]

X_test_h1.shape, y_test_h1.shape, X_test_h.shape, y_test_h.shape, X_val_h1.shape, y_val_h1.shape

## Modality 2

In [ ]:
X_train_s = np.load('SIPAKMED/features.npy')
y_train_s = np.load('SIPAKMED/labels.npy')

X_train_s.shape, y_train_s.shape

In [ ]:


X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_train_s, y_train_s, test_size=0.2, random_state=42)
X_train_s, X_val_s, y_train_s, y_val_s = train_test_split(X_train_s, y_train_s, test_size=0.2, random_state=42)

X_train_s.shape,X_test_s.shape, y_train_s.shape,y_test_s.shape, y_val_s.shape,y_val_s.shape

In [ ]:


def rotate_image(image, angle):
    """
    Rotate the image by the specified angle.
    """
    center = tuple(np.array(image.shape[1::-1]) / 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return rotated_image

def translate_image(image, tx, ty):
    """
    Translate the image by the specified translation parameters.
    """
    translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    translated_image = cv2.warpAffine(image, translation_matrix, image.shape[1::-1])
    return translated_image

def apply_gaussian_blur(image, kernel_size=3):
    """
    Apply Gaussian Blur to the image to reduce noise and improve generalization.
    """
    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    return blurred_image

# Augmentation parameters
rotation_angles = [20]
translations = [(5, 5)]
kernel_sizes = [3]  # Gaussian Blur kernel sizes

augmented_X_train = []
augmented_y_train = []

for image, label in zip(X_train_s, y_train_s):
    # Augment with rotations
    for angle in rotation_angles:
        rotated_image = rotate_image(image, angle)
        augmented_X_train.append(rotated_image)
        augmented_y_train.append(label)

    # Augment with translations
    for tx, ty in translations:
        translated_image = translate_image(image, tx, ty)
        augmented_X_train.append(translated_image)
        augmented_y_train.append(label)

    # Augment with Gaussian Blur
    for kernel_size in kernel_sizes:
        blurred_image = apply_gaussian_blur(image, kernel_size)
        augmented_X_train.append(blurred_image)
        augmented_y_train.append(label)

# Convert lists to numpy arrays
augmented_X_train = np.array(augmented_X_train)
augmented_y_train = np.array(augmented_y_train)

# Shuffle the data
shuffle_indices = np.random.permutation(len(augmented_X_train))
augmented_X_train = augmented_X_train[shuffle_indices]
augmented_y_train = augmented_y_train[shuffle_indices]
augmented_X_train.shape, augmented_y_train.shape




In [ ]:
random_indices = np.random.choice(7773, 5421, replace=False)

augmented_X_train = augmented_X_train[random_indices]
augmented_y_train = augmented_y_train[random_indices]

augmented_X_train.shape, augmented_y_train.shape


In [ ]:
X_train_s = np.concatenate((X_train_s, augmented_X_train), axis=0)
y_train_s = np.concatenate((y_train_s, augmented_y_train), axis=0)
X_train_s.shape, y_train_s.shape

In [ ]:
#del augmented_X_train

In [ ]:


# Normalize from [0, 255] to [0, 1]
X_train_s = X_train_s.astype(np.float32) / 255.0
X_test_s = X_test_s.astype(np.float32) / 255.0
X_val_s = X_val_s.astype(np.float32) / 255.0

X_train_s.shape, X_test_s.shape, X_val_s.shape

## Modality 3

In [ ]:
X_train_multi = np.load('X_train_img_BRCA_updated.npy')
X_val_multi = np.load('X_val_img_BRCA_updated.npy')
X_test_multi = np.load('X_test_img_BRCA_updated.npy')

y_train_multi = np.load('y_train_tab_BRCA_updated.npy')
y_val_multi = np.load('y_val_tab_BRCA_updated.npy')
y_test_multi = np.load('y_test_tab_BRCA_updated.npy')

X_train_multi.shape, X_val_multi.shape, X_test_multi.shape, y_train_multi.shape, y_val_multi.shape, y_test_multi.shape

## Modality 4

In [ ]:
X_train_m1 = np.load('X_train_MORT_MIMIC3_updated.npy')
y_train_m1 = np.load('y_train_MORT_MIMIC3_updated.npy')
X_test_m1 = np.load('X_test_MORT_MIMIC3_updated.npy')
y_test_m1 = np.load('y_test_MORT_MIMIC3_updated.npy')

X_val_m1 = np.load('X_val_MORT_MIMIC3_updated.npy')
y_val_m1 = np.load('y_val_MORT_MIMIC3_updated.npy')

X_train_m1.shape, y_train_m1.shape, X_test_m1.shape, y_test_m1.shape, X_val_m1.shape, y_val_m1.shape


In [ ]:
## HySAM



class GlobalMinPooling2D(layers.Layer):
    def __init__(self, **kwargs):
        super(GlobalMinPooling2D, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.reduce_min(inputs, axis=[1, 2])

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

    def get_config(self):
        config = super(GlobalMinPooling2D, self).get_config()
        return config

class MACFusion(tf.keras.layers.Layer):
    def __init__(self, units, activation='sigmoid', fractal_dim=1.8, learnable_curvature=True, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = activation
        self.fractal_dim = fractal_dim
        self.learnable_curvature = learnable_curvature

    def build(self, input_shape):
        feature_dim = input_shape[0][-1]
        self.feature_dim = feature_dim

        self.global_avg_pooling = layers.GlobalAveragePooling2D()
        self.global_max_pooling = layers.GlobalMaxPooling2D()
        self.global_min_pooling = GlobalMinPooling2D()

        self.global_attention = layers.Dense(units=self.units, activation=self.activation)
        self.global_scale1 = self.add_weight(shape=(1, 1, 1, 1), initializer=tf.keras.initializers.HeNormal(), trainable=True, name='global_scale1')
        self.global_scale2 = self.add_weight(shape=(1, 1, 1, 1), initializer=tf.keras.initializers.HeNormal(), trainable=True, name='global_scale2')
        
        # Quantum Components
        self.psi_real = self.add_weight(shape=(feature_dim, self.units),
                                        initializer='glorot_uniform', name='psi_real')
        self.psi_imag = self.add_weight(shape=(feature_dim, self.units),
                                        initializer='glorot_uniform', name='psi_imag')
        self.S = self.add_weight(shape=(self.units,),
                                 initializer=tf.keras.initializers.Constant(self.fractal_dim),
                                 constraint=MinMaxNorm(1.0, 2.0),
                                 name='fractal_scaling')

        # Hyperbolic Dual Genmetry Components
        self.curvature_log = self.add_weight(shape=(1,), initializer='he_normal',
                                             trainable=self.learnable_curvature,
                                             name='curvature_log')
        self.lorentz_alpha = self.add_weight(shape=(1,), initializer='he_normal', name='lorentz_scale')
        self.hyper_beta = self.add_weight(shape=(feature_dim,), initializer='he_normal', name='beta_scale')
        
        self.hyper_gate = self.add_weight(shape=(1,), initializer='ones', name='hyper_gate')
        self.hyper_gate1 = self.add_weight(shape=(1,), initializer='ones', name='hyper_gate1')

        # Cross-modality Gates
        self.cross_gate = self.add_weight(shape=(feature_dim,), initializer='ones', name='cross_gate')
        self.output_gate = self.add_weight(shape=(feature_dim,), initializer='ones', name='output_gate')

    def compute_curvature(self): # Compute leranable curvature read Eqs. 4 and 7 
        return tf.clip_by_value(tf.exp(self.curvature_log), 0.1, 10.0)

    # MLE of LIL block as for Eq. 7
    def lorentz_proj(self, x1, x2, c):

        space_part1 = x1
        space_part2 = x2
        norm_sq1 = tf.reduce_sum(x1 ** 2, axis=-1, keepdims=True)  #tf.reduce_sum(tf.square(x1), axis=-1, keepdims=True)
        norm_sq2 = tf.reduce_sum(x2 ** 2, axis=-1, keepdims=True) #tf.reduce_sum(tf.square(x2), axis=-1, keepdims=True)
        time1 = tf.sqrt(1 + c * norm_sq1) / c
        time2 = tf.sqrt(1 + c * norm_sq2) / c
        return tf.concat([time2, space_part1], axis=-1), tf.concat([time1, space_part2], axis=-1)
        '''
        space_part = x
        norm_sq = tf.reduce_sum(x ** 2, axis=-1, keepdims=True)
        time_part = tf.sqrt(1 + c * norm_sq) / c
        return tf.concat([time_part, space_part], axis=-1)

        '''

    '''def poincare_proj(self, x1, x2, c):
        norm1 = tf.norm(x1, axis=-1, keepdims=True) #+ 1e-6
        norm2 = tf.norm(x2, axis=-1, keepdims=True) #+ 1e-6
        scale1 = tf.tanh(tf.sqrt(c) * norm1) / norm1
        scale2 = tf.tanh(tf.sqrt(c) * norm2) / norm2
        return x1 * scale1, x2 * scale2
        '''
    # MPE of PIL block as for Eq. 4
    def poincare_proj(self, x1, x2, c):
        norm1 = tf.norm(x1, axis=-1, keepdims=True) #+ 1e-6
        norm2 = tf.norm(x2, axis=-1, keepdims=True) #+ 1e-6
        scale1 = tf.tanh(tf.sqrt(c) * norm1) * (x1 / (norm1 + 1e-6)) #tf.tanh(tf.sqrt(c) * norm1) / norm1
        scale2 = tf.tanh(tf.sqrt(c) * norm2) * (x2 / (norm2 + 1e-6))  #tf.tanh(tf.sqrt(c) * norm2) / norm2
        return x1 * scale1, x2 * scale2

    def call(self, inputs):
        input1, input2 = inputs

        
        input_shape = tf.shape(input1)
        
        #flattened_inputs1 = tf.reshape(input1, [-1, input_shape[-1]])  # Flatten along the last axis

        # Feature flattening
        x1_flat = tf.reduce_mean(input1, axis=[1, 2]) if len(input1.shape) == 4 else input1
        x2_flat = tf.reduce_mean(input2, axis=[1, 2]) if len(input2.shape) == 4 else input2

        ## Use DCT for capturing frequency information
        x1_flat = tf.signal.dct(x1_flat, type=2, norm='ortho')
        x2_flat = tf.signal.dct(x2_flat, type=2, norm='ortho')

        

        #. Multimodal Hyperbolic Dual-Geometry Attention (MHDGA)
        c = self.compute_curvature()
        adjusted_S = self.S * tf.sigmoid(c)
        adjusted_c = c * tf.reduce_mean(tf.sigmoid(self.S))

        
        # MQIA for estimating quantum states for each modality input
        psi1 = tf.complex(tf.matmul(x1_flat, self.psi_real), tf.matmul(x1_flat, self.psi_imag))
        psi2 = tf.complex(tf.matmul(x2_flat, self.psi_real), tf.matmul(x2_flat, self.psi_imag))

        
        # Mutual Guidance for MQIA through MLE of LIL block for each modality input as self.lorentz_proj() used in LIL block as for Eq. 9
        h_psi1, h_psi2 = self.lorentz_proj(tf.math.real(psi1), tf.math.real(psi2), adjusted_c)
        prob1 = tf.math.real(psi1 * tf.math.conj(psi1)) #+ 1e-8
        prob2 = tf.math.real(psi2 * tf.math.conj(psi2)) #+ 1e-8
        lorentz_factor1 = tf.norm(h_psi1[..., 1:], axis=-1, keepdims=True)
        lorentz_factor2 = tf.norm(h_psi2[..., 1:], axis=-1, keepdims=True)

        ## Attention maps learned from MQIA for each modality input as for Eq. 9
        
        scaled_prob1 = prob1 * tf.pow(lorentz_factor1, adjusted_S - 2)  
        scaled_prob2 = prob2 * tf.pow(lorentz_factor2, adjusted_S - 2)
        
        fractal_att1 = tf.nn.softmax(scaled_prob1, axis=-1)
        fractal_att2 = tf.nn.softmax(scaled_prob2, axis=-1)

        
        # Multimodal Hyperbolic Dual-Geometry Attention (MHDGA)
        h_lorentz1, h_lorentz2 = self.lorentz_proj(x1_flat, x2_flat, adjusted_c) # for LIL block for hyperbloc space computation as for Eq. 7
        h_poincare1, h_poincare2 = self.poincare_proj(x1_flat, x2_flat, adjusted_c) # for PIL block for hyperbloc space computation as for Eq. 5

        ## Ref. Eq. 6 to estimate times and frequency components for LIL:
        t1, x1 = h_lorentz1[..., :1], h_lorentz1[..., 1:]
        t2, x2 = h_lorentz2[..., :1], h_lorentz2[..., 1:]


        ## Bais estimation from MQIA for LIL block to generate attention maps see Eq. 8:
        quantum_bias1 = tf.matmul(fractal_att1, tf.transpose(self.psi_real))
        quantum_bias2 = tf.matmul(fractal_att2, tf.transpose(self.psi_real))


        

        # Lorentz attention weights is computed via Minkowski inner product (see Eq. 8):

        lorentz_att1 = self.lorentz_alpha * (-t1**2 + tf.reduce_sum((x1 + quantum_bias1)**2 * self.hyper_beta, axis=-1, keepdims=True))
        lorentz_att2 = self.lorentz_alpha * (-t2**2 + tf.reduce_sum((x2 + quantum_bias2)**2 * self.hyper_beta, axis=-1, keepdims=True))

        # Poincare attention weights is computed (see Eq. 5):
        poincare_att1 = tf.reduce_sum(h_poincare1 * x1_flat, axis=-1, keepdims=True)
        poincare_att2 = tf.reduce_sum(h_poincare2 * x2_flat, axis=-1, keepdims=True)

        #gate_factor = tf.tanh(self.hyper_gate)
        gate_factor = self.hyper_gate
        gate_factor1 = self.hyper_gate1

        # dual-geometry attention maps (see Eq. 3):
        hyper_att1 = tf.nn.sigmoid(gate_factor * lorentz_att1 + gate_factor1 * poincare_att1)
        hyper_att2 = tf.nn.sigmoid(gate_factor * lorentz_att2 + gate_factor1 * poincare_att2)

        # Reshape attention maps
        def expand(x, target_shape):
            if len(target_shape) == 4:
                x = tf.expand_dims(tf.expand_dims(x, 1), 1)
                return tf.tile(x, [1, target_shape[1], target_shape[2], 1])
            return x

        fractal_att1 = expand(fractal_att1, input1.shape)
        fractal_att2 = expand(fractal_att2, input1.shape)
        
        hyper_att1 = expand(hyper_att1, input1.shape)
        hyper_att2 = expand(hyper_att2, input2.shape)
        
        
        cross_gate = tf.reshape(self.cross_gate, (1, 1, 1, -1))
        output_gate1 = tf.reshape(self.output_gate, (1, 1, 1, -1))
        output_gate2 = tf.reshape(self.output_gate, (1, 1, 1, -1))
        output_gate3 = tf.reshape(self.output_gate, (1, 1, 1, -1))
        output_gate4 = tf.reshape(self.output_gate, (1, 1, 1, -1))

        #hyper_att = hyper_att1 + hyper_att2
        
        # Cross Fusion (see Eqs. 1, 2, 10):
        fractal_out1 = input2 * cross_gate * fractal_att1 #* attention1 #* hyper_att1
        hyper_out1 = input2 * cross_gate * hyper_att1 #* attention1
        
        fractal_out2 = input1 * cross_gate * fractal_att2 #* attention2 #* hyper_att1
        hyper_out2 = input1 * cross_gate * hyper_att2 #* attention2
        
        #  Multimodal Attention Fusion Gating Block:
        return ((output_gate1 * fractal_out2) + (output_gate2 * hyper_out2)), ((output_gate3 * fractal_out1) + (output_gate4 * hyper_out1)) #+ (output_gate4 * attn1) + (output_gate4 * attn2)


In [ ]:
## EXtending MHCF for devsiing SIR module
def multi_kernel_groupwise_conv1(x, filters, groups=8, strides=2):
    

    # Multi-Kernel Parallel Convolutions with Different Receptive Fields
    conv1x1 = layers.Conv2D(filters // 4, kernel_size=1, groups=groups, padding="same")(x)
    conv3x3 = layers.DepthwiseConv2D(kernel_size=3, padding="same")(x)
    conv5x5 = layers.DepthwiseConv2D(kernel_size=5, padding="same")(x)
    conv_dilated = layers.DepthwiseConv2D(kernel_size=3, dilation_rate=2, padding="same")(x)

    # Feature Fusion via Concatenation
    x1 = layers.Concatenate()([conv1x1, conv3x3, conv5x5, conv_dilated])

    # Channel Shuffle for Better Feature Mixing
    def channel_shuffle(x, groups):
        batch, height, width, channels = tf.unstack(tf.shape(x))
        x = tf.reshape(x, [-1, height, width, groups, channels // groups])
        x = tf.transpose(x, [0, 1, 2, 4, 3])
        x = tf.reshape(x, [-1, height, width, channels])
        return x

    x1 = layers.Lambda(lambda x: channel_shuffle(x, groups))(x1)

    # Depthwise + Grouped Convolutions Hybrid
    x1 = layers.DepthwiseConv2D(kernel_size=3, padding="same")(x1)
    x1 = layers.Conv2D(filters, kernel_size=1, groups=groups, padding="same")(x1)

    # Downsampling x1
    x1 = layers.DepthwiseConv2D(kernel_size=3, strides=strides, padding="same")(x1)

    # **Fix: Ensure x has the same number of channels as x1**
    x = layers.Conv2D(filters, kernel_size=1, groups=groups, padding="same")(x)
    x = layers.DepthwiseConv2D(kernel_size=3, strides=strides, padding="same")(x)

    # Residual Connection
    x = layers.Add()([x, x1])
    x = layers.Activation('gelu')(x)  # GELU activation for better convergence
    
    return x




# MHCF for EMRC block
def multi_kernel_groupwise_conv3(x, filters, groups=8, strides=2, use_se=True):
    # GPC
    conv1x1 = layers.Conv2D(filters, kernel_size=1, groups=groups, strides=strides, padding="same", use_bias=False)(x)
    conv1x1 = layers.BatchNormalization()(conv1x1)

    # DDC
    conv3x3 = layers.DepthwiseConv2D(kernel_size=3, dilation_rate=1,strides=strides,  padding="same", use_bias=False)(x)
    
    #conv3x3 = layers.GaussianBlur(kernel_size=3, sigma=1.0)(conv3x3)
    #if strides==2:
     #   conv3x3 = layers.AveragePooling2D(2, 2, padding="same")(conv3x3)

    conv3x3 = layers.BatchNormalization()(conv3x3)

    # DWC
    conv5x5 = layers.DepthwiseConv2D(kernel_size=5, strides=strides, padding="same", use_bias=False)(x)
    conv5x5 = layers.BatchNormalization()(conv5x5)

    # Concatenation and 1x1 Fusion
    x1 = layers.Concatenate()([conv1x1, conv3x3, conv5x5])

    def channel_shuffle(x, groups):
        batch, height, width, channels = tf.unstack(tf.shape(x))
        x = tf.reshape(x, [-1, height, width, groups, channels // groups])
        x = tf.transpose(x, [0, 1, 2, 4, 3])
        x = tf.reshape(x, [-1, height, width, channels])
        return x

    x1 = layers.Lambda(lambda x: channel_shuffle(x, groups))(x1)

    
    x1 = layers.Conv2D(filters, kernel_size=1, groups=groups, padding="same", use_bias=False)(x1)
    
    x1 = layers.BatchNormalization()(x1)

    # Residual Connection
    x = layers.Conv2D(filters=filters, kernel_size=1, strides=strides, groups= 8, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Add()([x, x1])
    x = layers.Activation('gelu')(x)  # GELU activation for better convergence

    return x


# Single branch of EMRC for each modality input, such as x_i

def RGSA(x, filters, strides=(1, 1), use_projection=False):
    shortcut = x  # Save the original input for residual connection

    # First MHCF
    x = multi_kernel_groupwise_conv3(x, filters=filters, groups=8, strides=strides)

    # Normalization and Activation
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Second MHCF
    x = multi_kernel_groupwise_conv3(x, filters=filters, groups=8, strides=(1, 1))  # Strides=1 to avoid mismatch
    x = layers.Conv2D(filters, kernel_size=(1, 1), padding="same")(x)  # Pointwise conv for channel mixing
    x = layers.BatchNormalization()(x)

    # Adjust Shortcut if Needed (Ensure Matching Shape)
    if strides != (1, 1) or use_projection:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, groups=8, padding="same")(shortcut)  # Downsampling shortcut
        shortcut = layers.BatchNormalization()(shortcut)

    #x, shortcut = MACFusion(units=filters)([x, shortcut])
    # Residual Connection (Ensure Same Shape)
    x = layers.Add()([x, shortcut])

    # Final Activation
    x = layers.Activation('relu')(x)

    return x


In [ ]:
def EMRC(x1, x2, filters, strides):
    """
    Efficient Multiscale Residual Convolution (EMRC) block.

    Applies the RGSA module to each of the two input feature maps,
    using a projection shortcut when the spatial resolution or channel
    count changes.

    Args:
        x1: First input tensor.
        x2: Second input tensor.
        filters: Number of output channels for RGSA.
        strides: Stride to use in RGSA.

    Returns:
        A tuple (x1, x2) of output tensors after RGSA.
    """
    # (optional) fast input check
    if not (isinstance(filters, int) and filters > 0):
        raise ValueError(f"`filters` must be a positive int, got {filters}")
    if not (isinstance(strides, int) and strides > 0):
        raise ValueError(f"`strides` must be a positive int, got {strides}")

    x1 = RGSA(x1, filters=filters, strides=strides, use_projection=True)
    x2 = RGSA(x2, filters=filters, strides=strides, use_projection=True)
    return x1, x2

In [ ]:
def info_fusion(x_1, x_2, filter):
    x11 = multi_kernel_groupwise_conv1(x_1, filters=filter, groups=8, strides=(2,2))
    x21 = multi_kernel_groupwise_conv1(x_2, filters=filter, groups=8, strides=(2,2))

    return x11, x21

def info_fusion1(x_1, x_2, filter):
    x11 = multi_kernel_groupwise_conv1(x_1, filters=filter, groups=8, strides=(1,1))
    x21 = multi_kernel_groupwise_conv1(x_2, filters=filter, groups=8, strides=(1,1))

    return x11, x21

In [ ]:
def SIR(x1, x2, f):
    if f == 64:
        x111, x211 = info_fusion(x1, x2, 2 * f)
        x111, x211 = info_fusion(x111, x211, 4 * f)
        x111, x211 = info_fusion(x111, x211, 8 * f)
        return x111, x211

    elif f == 128:
        x111, x211 = info_fusion(x1, x2, 2 * f)
        x111, x211 = info_fusion(x111, x211, 4 * f)
        return x111, x211

    elif f == 256:
        x111, x211 = info_fusion(x1, x2, 2 * f)
        return x111, x211

    elif f == 512:
        x111, x211 = info_fusion1(x1, x2, f)
        return x111, x211

    else:
        raise ValueError(f"Unsupported fusion factor: {f}")

In [ ]:
## HySAM
def PCMFA(x1, x2, f):
    
    x1, x2 = MACFusion(units=f)([x1, x2]) # First PCMFA
    x1, x2 = MACFusion(units=f)([x1, x2]) # Second PCMFA

    return x1, x2

In [ ]:
## HyFuse

def EHF1(x1, x2, f, s):
    x1, x2 = EMRC(x1, x2, f, s) # Call EMRC module
    x1, x2 = PCMFA(x1, x2, f) # # Call PCMFA module
    return x1, x2    

In [ ]:
from tensorflow.keras.initializers import Ones, Constant

## LLF 

class LF(layers.Layer):
    def build(self, input_shape):
        # input_shape: list of two shapes, each (B, H, W, C)
        # we use a single scalar per tensor (broadcastable to all channels)
        

        self.global_scale1 = self.add_weight(
            name="global_scale1",
            shape=(1,1,1,1),
            initializer=Ones(),      # starts at 1.0
            trainable=True
        )
        # or, if you want a tiny deviation around 1:
        self.global_scale2 = self.add_weight(
            name="global_scale2",
            shape=(1,1,1,1),
            initializer=Constant(value=1.0),  
            # this gives values ~ N(1,0.01)
            trainable=True
        )

    def call(self, inputs):
        x1, x2 = inputs
        # scale each branch by its learnable scalar
        x1_scaled = x1 * self.global_scale1
        x2_scaled = x2 * self.global_scale2
        # concatenate along channel axis
        return layers.Concatenate(axis=-1)([x1_scaled, x2_scaled])

In [ ]:
## HyFuse

def EHF2(x1, x2, f, s):
    x1, x2 = EMRC(x1, x2, f, s) # Call EMRC module
    
    x1 = layers.BatchNormalization()(x1)
    x2 = layers.BatchNormalization()(x2)
    
    x1, x2 = PCMFA(x1, x2, f) # Call PCMFA module
    
    x111, x211 = SIR(x1, x2, f) ## Call SIR module

    #x1 = layers.Concatenate(axis = -1)([x1, x2])
    
    x1 = LF()([x1, x2])  # Use Learnable Late Fusion (LF) for capturing multimodal information
    
    x1 = layers.Conv2D(f, kernel_size=(1, 1), groups=8, 
                       padding="same")(x1) 
    
    return x1, x111, x211    

In [ ]:
## HyFuse

def EHF(x1, x2, filters, strides1, strides2):
    """
    Efficient Hybrid Fusion (EHF) block.

    - If filters == 64, applies two rounds of EHF1 at strides2 and returns (x1, x2).
    - If filters >= 128, applies EHF1 at strides1 then EHF2 at strides2 and returns (x1, x2, x11, x22).
    """
    if filters == 64:
        x1, x2 = EHF1(x1, x2, filters, strides2)
        x1, x2 = EHF1(x1, x2, filters, strides2)
        return x1, x2

    elif filters == 128:
        x1, x2 = EHF1(x1, x2, filters, strides1)
        x1, x111, x211 = EHF2(x1, x2, filters, strides2)
        return x1, x111, x211

    elif filters == 256:
        x1, x2 = EHF1(x1, x2, filters, strides1)
        x1, x111, x211 = EHF2(x1, x2, filters, strides2)
        return x1, x111, x211

    elif filters == 512:
        x1, x2 = EHF1(x1, x2, filters, strides1)
        x1, x111, x211 = EHF2(x1, x2, filters, strides2)
        return x1, x111, x211

    else:
        raise ValueError(f"Unsupported filter size: {filters}. Expected 64 or >=128.")


In [ ]:
## MSIL

def residual_GLC_branch1(inputs1, inputs2, inputs3, inputs4):

    # Modality input 1
    x1 = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs1)
    x1 = BatchNormalization()(x1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x1)

    # Modality input 2
    x2 = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs2)
    x2 = BatchNormalization()(x2)
    x2 = tf.keras.layers.Activation('relu')(x2)
    x2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x2)

    
    x1, x2 = EHF(x1, x2, filters=64, strides1=1, strides2=1)

    x1, x111, x211 = EHF(x1, x2, filters = 128, strides1=1, strides2=2)


    # Modality input 3
    x3 = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs3)
    x3 = BatchNormalization()(x3)
    x3 = tf.keras.layers.Activation('relu')(x3)
    x3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x3)

    x2 = multi_kernel_groupwise_conv3(x3, filters=256, groups=8, strides=2)
    
    x1, x1111, x2111 = EHF(x1, x2, filters = 256, strides1=1, strides2=2)

    
    # Modality input 4
    x4 = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs4)
    x4 = BatchNormalization()(x4)
    x4 = tf.keras.layers.Activation('relu')(x4)
    x4 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x4)
    
    
    x2 = multi_kernel_groupwise_conv3(x4, filters=512, groups=8, strides=2)
    x2 = multi_kernel_groupwise_conv3(x2, filters=512, groups=8, strides=2)
    
    #x2 = RGSA(x2, filters=512, strides=(2, 2), use_projection=True)

    x1, x11111, x21111 = EHF(x1, x2, filters = 512, strides1=1, strides2=2)
    
    
    return x1, x111, x211, x1111, x2111, x11111, x21111

In [ ]:
'''class CIndexCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Predict risk scores on the full validation set
        pred = self.model.predict(X_val_multi, verbose=0).reshape(-1)
        #risk_scores = preds[2].reshape(-1)
        durs = y_val_tab[:, 0]
        evts = y_val_tab[:, 1].astype(bool)
        ci = concordance_index(durs, -pred, evts)
        logs = logs or {}
        logs['val_c_index'] = ci
        print(f" — val_c_index: {ci:.4f}")
'''

class CIndexCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_inputs, val_cox_y):
        """
        val_inputs: list of your four validation inputs,
                    e.g. [X_val_s, X_val_h1, X_val_multi, X_val_m1]
        val_cox_y:  the (n,2) array of [durations, events] for the Cox head
        """
        super().__init__()
        self.val_inputs = val_inputs
        self.val_cox_y   = val_cox_y

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        # 1) predict returns a list of outputs, one per head
        preds = self.model.predict(self.val_inputs, batch_size = 4, verbose=0)
        # 2) pick out the Cox‐head predictions (assuming it's the 3rd head: index 2)
        risk_scores = preds[2].reshape(-1)

        # 3) pull durations/events
        durations = self.val_cox_y[:, 0]
        events    = self.val_cox_y[:, 1].astype(bool)

        # 4) compute C‐index
        ci = concordance_index(durations, -risk_scores, events)
        logs['val_c_index'] = ci

        print(f" — val_c_index: {ci:.4f}")


#cidx_cb = CIndexCallback()

val_inputs = [X_val_s, X_val_h1, X_val_multi, X_val_m1]
# the Cox‐head targets for val:
val_cox_y = y_val_multi  # shape (n_val, 2)

cidx_cb = CIndexCallback(val_inputs, val_cox_y)


In [ ]:
def neg_log_partial_likelihood(y_true, y_pred):
    eps = 1e-8

    # unpack
    durations = y_true[:, 0]
    events    = y_true[:, 1]

    # sort by descending time
    order = tf.argsort(durations, direction="DESCENDING")
    p = tf.gather(tf.reshape(y_pred, [-1]), order)

    # clip the raw predictions to a safe window before exp
    p = tf.clip_by_value(p, -10.0, 10.0)

    # hazard ratio
    hr = tf.exp(p)
    hr = tf.clip_by_value(hr, eps, 1e6)

    # cumulative hazard
    cum_h = tf.cumsum(hr) + eps
    log_cum = tf.math.log(cum_h)

    diff = p - log_cum
    loss = -tf.reduce_sum(diff * tf.gather(events, order))

    return loss / (tf.reduce_sum(events) + eps)


In [ ]:
import tensorflow as tf

class ConcordanceIndex(tf.keras.metrics.Metric):
    def __init__(self, name='c_index', **kwargs):
        super().__init__(name=name, **kwargs)
        self.y_true_vals = []
        self.y_pred_vals = []

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.y_true_vals.append(tf.reshape(y_true, [-1]))
        self.y_pred_vals.append(tf.reshape(y_pred, [-1]))

    def result(self):
        y_true = tf.concat(self.y_true_vals, axis=0)
        y_pred = tf.concat(self.y_pred_vals, axis=0)
        return self._concordance_index(y_true, y_pred)

    def reset_states(self):
        self.y_true_vals = []
        self.y_pred_vals = []

    def _concordance_index(self, y_true, y_pred):
        n = tf.shape(y_true)[0]
        i = tf.range(n)
        j = tf.range(n)
        ii, jj = tf.meshgrid(i, j)
        mask = tf.math.greater(y_true[ii], y_true[jj])
        pred_diff = y_pred[ii] - y_pred[jj]
        concordant = tf.math.greater(pred_diff, 0)
        ties = tf.math.equal(pred_diff, 0)
        concordant = tf.cast(concordant, tf.float32)
        ties = tf.cast(ties, tf.float32)
        concordant = tf.where(mask, concordant, tf.zeros_like(concordant))
        ties = tf.where(mask, ties, tf.zeros_like(ties))
        total = tf.reduce_sum(tf.cast(mask, tf.float32))
        return (tf.reduce_sum(concordant) + 0.5 * tf.reduce_sum(ties)) / (total + tf.keras.backend.epsilon())


In [ ]:
#def build_resnet18(input_shape=(128, 128, 3), num_classes=2):
def build_model():

    input_shape=(128, 128, 3)
    inputs1 = Input(shape=input_shape) # Modality input 1
    inputs2 = Input(shape=input_shape) # Modality input 2
    inputs3 = Input(shape=input_shape) # Modality input 3
    inputs4 = Input(shape=input_shape) # Modality input 4
    
    import tensorflow.keras.layers as L
    
    #input_data = Input(shape=input_shape, name='input_data')
    # Initial convolutional layer
    
    #x1, x2 = residual_GLC_branch1(inputs1, inputs2)
    
    ## Call MSIL

    x1, x111, x211, x1111, x2111, x11111, x21111 = residual_GLC_branch1(inputs1, inputs2, inputs3, inputs4)
    #print('x:',x.shape)
    
    

    con1 = tf.keras.layers.Concatenate(axis=-1)([x111, x1111, x11111])
    con2 = tf.keras.layers.Concatenate(axis=-1)([x211, x2111, x21111])

    con = tf.keras.layers.Concatenate(axis=-1)([x1, con1, con2])

    
    
    
    #con = tf.keras.layers.Concatenate(axis=-1)([x1, x111, x1111, x11111, x211, x2111, x21111])
    
    con = tf.keras.layers.Dropout(0.25)(con, training = True)  ## MCD ####
    
    ## HMML
    x = GlobalAveragePooling2D()(con)
    
    
    outputs1 = Dense(5, activation='softmax')(x) # Output layer for modality input 1
    outputs2 = Dense(7, activation='softmax')(x) # Output layer for modality input 2
    outputs3 = Dense(1, activation="linear")(x) # Output layer for modality input 3
    outputs4 = Dense(2, activation='sigmoid')(x) # Output layer for modality input 4
    
    # Create the model
    model = Model([inputs1, inputs2, inputs3, inputs4], [outputs1, outputs2, outputs3, outputs4])
    #return model
    #print(model.summary())

    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.optimizers.schedules import ExponentialDecay

    initial_gamma1 = 0.25
    initial_gamma2 = 0.25
    initial_gamma3 = 0.25
    
    
    
    from tensorflow.keras.optimizers import Adam

    #opt = Adam(learning_rate=1e-4, clipnorm=1.0)
    
    #opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.9, epsilon=None, amsgrad=False)
    
    # Compile the model with the custom optimizer
    model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy', neg_log_partial_likelihood, 
                                          'binary_crossentropy'],
                  loss_weights=[initial_gamma1, initial_gamma2, initial_gamma3, (1 -  (initial_gamma1 + initial_gamma2 + initial_gamma3))],
                  metrics={
        'h':    ['accuracy', tf.keras.metrics.AUC(name='auc')],
        's':    ['accuracy', tf.keras.metrics.AUC(name='auc')],
        'multi':[ConcordanceIndex(name='c_index')],
        'm1':   ['accuracy', tf.keras.metrics.AUC(name='auc')]
    }
                 )
       
    return model

In [ ]:
model = build_model()

print(model.summary())

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
def checkpoint_callback():

    checkpoint_filepath = 'best1_model.keras'

    model_checkpoint_callback= ModelCheckpoint(filepath=checkpoint_filepath,
                           save_weights_only=False,
                           #frequency='epoch',
                           monitor='val_loss',
                           save_best_only=True,
                            mode='min',
                           verbose=1)

    return model_checkpoint_callback

def early_stopping(patience):
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=60, verbose=1)
    return es_callback



from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=50, verbose = 1, min_lr=0.000001)

checkpoint_callback = checkpoint_callback()

early_stopping = early_stopping(patience=100)
callbacks = [#cidx_cb, 
             checkpoint_callback, early_stopping, reduce_lr]



In [ ]:
# Fit the model with callbacks
history = model.fit([X_train_s, X_train_h, X_train_multi, X_train_m1], [y_train_s, y_train_h, y_train_multi, y_train_m1],
                    epochs=200,
                    validation_data=([X_val_s, X_val_h1, X_val_multi, X_val_m1], [y_val_s, y_val_h1, y_val_multi, y_val_m1]), verbose=1, 
                    shuffle=True, #batch_size=40,
                    callbacks=callbacks) 

In [ ]:
import numpy as np
import tensorflow as tf
import random

# 1) Generate or define your five seeds
import random

# 1) Generate five random 32-bit integer seeds
#    (you can set a seed here if you want reproducible seed-generation)
seeds = np.random.randint(0, 2**31 - 1, size=5).tolist()
print("Using seeds:", seeds)


# 2) Prepare empty lists for each modality
modalities = ['Dermoscopy', 'Cytology', 'ECG', 'EHR']
stats = {
    mod: {
        'accs': [],
        'metrics': []  # 'auc' or 'c_index' depending on modality
    } for mod in modalities
}


for seed in seeds:
    # ── reproducibility ───────────────────────────────────────────
    tf.keras.backend.clear_session()
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    # ── build & compile ───────────────────────────────────────────
    model = build_model()  # your model builder
    

    # ── fit ───────────────────────────────────────────────────────
    model.fit(
        [X_train_h, X_train_s, X_train_multi, X_train_m1],
        [y_train_h, y_train_s, y_train_multi, y_train_m1],
        epochs=200,
        validation_data=(
            [X_val_h1, X_val_s, X_val_multi, X_val_m1],
            [y_val_h1, y_val_s, y_val_multi, y_val_m1]
        ),
        verbose=0,
        shuffle=True,
        callbacks=callbacks
    )

    # ── evaluate ─────────────────────────────────────────────────
    names, results = model.metrics_names, model.evaluate(
        [X_test_h1, X_test_s, X_test_multi, X_test_m1],
        [y_test_h1, y_test_s, y_test_multi, y_test_m1],
        verbose=0
    )
    res = dict(zip(names, results))

    # ── collect per‐modality ──────────────────────────────────────
    # After model evaluation
    for mod in modalities:
        stats[mod]['accs'].append(res[f"{mod}_accuracy"])
        if mod == 'ECG':  # Assuming 'multi' corresponds to 'ECG'
            stats[mod]['metrics'].append(res[f"{mod}_c_index"])
        else:
            stats[mod]['metrics'].append(res[f"{mod}_auc"])


for mod in modalities:
    accs = np.array(stats[mod]['accs'])
    metrics = np.array(stats[mod]['metrics'])
    metric_name = 'C-Index' if mod == 'ECG' else 'AUC'
    print(f"Modality {mod.upper()}:")
    print(f"  ACC: {accs.mean():.4f} ± {accs.std():.4f}")
    print(f"  {metric_name}: {metrics.mean():.4f} ± {metrics.std():.4f}\n")
